In [1]:
library(magrittr)
library(MASS)

## Refactoring

Refactoring code is a very important concept in programming. To "refactor" means to change the implementation of some piece of code, without changing the functionality. 

If your goal with writing code is simply to "get something to work" (once), then refactoring is not useful. If, however, your goal is to write elegant code which can be used reliable, extended, and modified, then you will always need to refactor! 

In [2]:
# We will refactor the code from the "Functions in R" lecture. 

# Using the ideas from the slides, make "generate_data" a polymorphic function.
# It should operate on two classes: "univariate_params" and "multivariate_params"
# and two separate "methods" should be implemented for each of the classes 

In [35]:
# The implementation of the class is up to you! So write the params class to
# match your design. 

# beta should be c(1,2,1)
# mean of X's should be c(0,0,0)
# sd of X's should be c(.2,.5,.3)
# noise sd should be .5

generate_data <- function(params) UseMethod("generate_data")
generate_data.univariate_params <- function(params){
    X <- rnorm(params$N, 0, 1)
    eps <- rnorm(params$N, 0, params$noise)
    y <- params$beta*X + eps
    output <- list(X = X, y = y)
    class(output) <- "univariate_data"
    output
}
generate_data.multivariate_params <- function(params){
    params1 <- cbind(params$mean, params$sd)
    X <- cbind(apply(params1,1, function(p) rnorm(params$N, p[1],  p[2])))
    eps <- rnorm(params$N, 0, params$noise)
    y <- X %*% params$beta + eps
    output <- list(X=X, y=y)
    class(output) <- "multivariate_data"
    output
}

calc_coef <- function(data) UseMethod("calc_coef")
calc_coef.univariate_data <- function(data) cov(data$X,data$y) / var(data$X)
calc_coef.multivariate_data <- function(data) solve(t(data$X) %*% data$X, t(data$X) %*% data$y) 

calc_se <- function(data, coef) UseMethod("calc_se")
calc_se.univariate_data <- function(data, coef) {
    n <- length(data$y)
    eps <- data$y - data$X * coef
    e_sd <- mean(eps^2)
    se <- sqrt(e_sd / (n*var(data$X)))
    se
}
calc_se.multivariate_data <- function(data, coef) {
    eps <- data$y - data$X %*% coef
    e_sd <- mean(eps^2)
    sqrt(diag(e_sd * solve(t(data$X) %*% data$X)))
}
                     
run_regression <- function(data){
    coef <- calc_coef(data)
    se <- calc_se(data, coef)
    list(coef=coef,se=se)
}             

eval_model <- function(model, params, conf = 1.96) {
    up <- model$coef + model$se * conf
    down <- model$coef - model$se * conf
    params$beta > down & params$beta < up
}

simulate <- function(params) {
    d <- generate_data(params)
    m <- run_regression(d)
    eval_model(m, params)
}

avg_simulations <- function(M, params) {
    inside <- sapply(1:M, function (x) simulate(params))
    rowMeans(inside)
}
                     
params_m <- list(mean=c(0,0,0),sd=c(.2,.5,.3),beta=c(1,2,1),noise=.5,N=100)
class(params_m) <- "multivariate_params"
data_m <- generate_data(params_m)
params_u <- list(mean=c(0),sd=c(.2),beta=c(2),noise=.5,N=100)
class(params_u) <- "univariate_params"
data_u <- generate_data(params_u)
model_m <- run_regression(data_m)
#eval_model(model_m,params_m)
simulate(params_m)
# avg_simulations(10,params_m)

TRUE
TRUE
TRUE


In [26]:
a <- avg_simulations(1000, 20, params)
stopifnot(round(a, 1) == .9)

ERROR: Error in avg_simulations(1000, 20, params): could not find function "avg_simulations"


In [ ]:
a <- avg_simulations(1000, 500, params)
stopifnot(a > .92)

In [ ]:











# BONUS
                     
# The calc_coef and calc_se functions, in their multivariate form, are 
# naturally a generalization, and work for univariate data do. 

# But let's pretend they're not. 

# Write them as polymorphic functions, such that when you simulation gets called
# with univariate data, the (from-the-slides) univariate formula gets called, 
# and the (from-your-exersizes) multivariate formula gets called when 
# multivariate parameters are given to the simulations. 

# Note: be creative in your design! Add more classes if you think it helps!
# Think about which functions you want to be class-agnostic and which to be
# class-aware! 